In [1]:
import numpy as np
import pandas as pd

top_2017 = pd.read_csv('../datasets/top2017.csv')
top_2018 = pd.read_csv('../datasets/top2018.csv')
top_2019 = pd.read_csv('../datasets/top2019.csv', encoding='mbcs', index_col=0)
top_2010_2019 = pd.read_csv('../datasets/top2010-2019.csv', encoding='mbcs', index_col=0)